# Plotting with pymovements

Pymovements provides a collection of built-in plotting functions 
to visualize gaze data in both temporal and spatial dimensions.

These functions make it easy to explore and present your data, 
from individual trial visualizations to aggregated participant-level analyses.

In this tutorial, you’ll learn how to:

- load and prepare a sample dataset for plotting
- compute the necessary properties
- plot gaze traces over time (traceplot)
- visualize fixations on a stimulus (scanpathplot)
- generate heatmaps showing gaze density (heatmap)
- plot the saccadic main sequence (main-sequence)
- style the plot

All examples use the small {doc}`ToyDataset <../reference/api/pymovements.datasets.ToyDataset>` that comes with pymovements.

## Loading and preprocessing data

In [ ]:
import polars as pl

import pymovements as pm

dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

# Convert the raw x and y coordinates in pixels to degrees of visual angle
dataset.pix2deg()

# Compute gaze velocities in dva/s from dva coordinates.
dataset.pos2vel('smooth')

## Plot Raw Samples

In [ ]:
# we will work with gaze data from the first recording
gaze = dataset.gaze[0]

# extract position_x and position_y from the list column
df = gaze.samples

df = df.with_columns([
    pl.col("position").list.get(0).alias("pos_x"),
    pl.col("position").list.get(1).alias("pos_y"),
])

# Assign back
gaze.samples = df

### Tsplot

The time series (ts) plot visualizes raw gaze samples directly from a Gaze object.
Each channel is plotted as a time series, allowing you to inspect the recorded signals before any processing or detection steps.

In [ ]:
pm.plotting.tsplot(
    gaze,
    # A separate trace is plotted for each channel.
    # It is plotted against sample index, not spatial axes.
    # Rapid jumps in pos_x (saccades) look almost vertical
    # slowly changing pos_y looks like gentle/horizontal waves.
    channels=['pos_x', 'pos_y'],
    # Set separate y-axis for each channel.
    share_y=False,
    line_color="darkblue")

### Traceplot

A traceplot visualizes the raw gaze samples. It represents eye movements as a continuous trajectory connecting gaze points, where each point corresponds to a specific position and moment in time. This helps reveal viewing patterns.

Traceplots are useful for:
- Verifying that gaze data have been parsed and aligned correctly.
- Exploring viewing behavior across conditions or participants.
- Identifying artifacts or data quality issues.

A basic traceplot can be created with only a `Gaze` object:

In [ ]:
pm.plotting.traceplot(gaze)

## Detecting and Visualizing Events

Eye-tracking data are typically segmented into events, i.e. fixations and saccades. Fixations represent moments when the eyes remain relatively still, allowing visual information to be processed, while saccades are the rapid movements between fixations that reposition the gaze. Detecting these events and computing their properties, such as fixation duration, saccade amplitude, and peak velocity, provides the foundation for analyzing visual behavior and understanding how participants explore a stimulus.

### Fixations

We can detect fixations by applying the I-VT or the I-DT method. 

The **I-VT (Velocity-Threshold Identification)** method distinguishes fixation and saccade points based on their point-to-point velocities. Each point is classified as a fixation if its velocity is below the specified threshold. Consecutive fixation points are then merged into a single fixation. A threshold of 20 degrees/second is commonly used as a default maximum value. Read more about the [IVT method in the documentation](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.ivt.html).

The **I-DT (Dispersion-Threshold Identification)** method finds fixations by grouping consecutive points within a maximum separation (dispersion) threshold and a minimum duration threshold. The algorithm slides a moving window across the data: if the dispersion within the window is below the threshold, the window represents a fixation and is gradually expanded until the dispersion exceeds the threshold.
Read more about our implementation of the [IDT method](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.idt.html).

We will use the I-DT algorith mwith different dispersion threshold values to create two differnet sets of fixation events.

**Key Parameters:**
- `dispersion_threshold`: Maximum dispersion allowed for fixation points. Default: 1.0 degree
- `name`: Custom name for the detected events

The `mininum_duration` default is 100 ms.

In [ ]:
# Detect fixations with stricter threshold (1.0 degrees)
dataset.detect('idt', dispersion_threshold=1.0, name='fixation_1.0_idt')

# Detect fixations with standard threshold (2.7 degrees)
dataset.detect('idt', dispersion_threshold=2.7, name='fixation_2.7_idt')

### Calculating Fixation Properties

The property `location` will be used for visualization purposes. It is added as a separate column named `location` in the events DataFrame, containing the centroid coordinates of each fixation. 

**Key Parameter:**
- `position_column`: Specifies which coordinate system to use for the property. By default, fixation centroids are computed in degrees of visual angle. To obtain fixation centroids in pixel coordinates, this parameter must be explicitly set to `pixel`. 

In [ ]:
# Compute fixation locations using pixel coordinates
dataset.compute_event_properties(("location", {'position_column': 'pixel'}))

### Creating the Scanpath Plot

A scanpath plot visualizes the sequence of fixations as circles, with size representing duration. In a complete scanpath plot, each fixation would have an arrow pointing to the next fixation in viewing order (this feature is currently not implemented).

In [ ]:
# show all unique event names in the gaze events frame
gaze.events.frame.select('name').unique().to_series().to_list()

In [ ]:
pm.plotting.scanpathplot(gaze, event_name='fixation_1.0_idt')

In [ ]:
pm.plotting.scanpathplot(gaze, event_name='fixation_2.7_idt')

We can create an enhanced visualization by overlaying the scanpath plot with the traceplot. This shows both the fixations, their duration, and the raw gaze trajectory.

In [ ]:
pm.plotting.scanpathplot(gaze, event_name='fixation_2.7_idt', add_traceplot=True)

### Heatmap Plotting

The heatmap visualizes the spatial distribution of gaze samples across the experiment screen. Each cell's color value reflects the cumulative time (in seconds) that the gaze samples were recorded at that position.

We can use the `heatmap` function from the `pymovements` library with the default values for `gridsize` (10x10), interpolation, and the colorbar.

In [ ]:
pm.plotting.heatmap(gaze)

Furthermore, we can customize various aspects of the heatmap plot, such as the gridsize, color map, and the labels.

In [ ]:
figure = pm.plotting.heatmap(
    gaze=gaze,
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation On',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
)

To better understand the effect of the `gridsize` parameter on the heatmap, we can turn off the interpolation. By doing this, we can clearly visualize the individual bins used to calculate the heatmap. With interpolation turned off, the heatmap will display the raw bin values rather than a smoothed representation.

In [ ]:
figure = pm.plotting.heatmap(
    gaze,
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation Off',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
    interpolation='none'
)

Increasing the `gridsize` parameter results in a finer grid and more detailed heatmap representation. With a higher grid size, we divide the plot into smaller bins, which can capture more nuances in the data distribution

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Higher Grid Size',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[25, 25]
)

### Detect Saccades and Compute Amplitude and Peak Velocity

Saccades are rapid eye movements that shift the point of fixation from one location to another. We detect saccades (or micro-saccades) from velocity gaze sequence. The `microsaccades` algorithm ([see documentation](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.microsaccades.html#microsaccades)) has a noise-adaptive velocity threshold parameter, which can be set explicitly.

In [ ]:
# detect saccades using the microsaccades algorithm

# Defaults:
# minimum_duration = 6 ms (or 6 samples if timesteps is None)
# threshold_factor = 5
dataset.detect_events('microsaccades', minimum_duration=6, threshold_factor=5)

In [ ]:
# compute amplitude and peak velocity of the detected saccades
dataset.compute_event_properties(['amplitude', 'peak_velocity'])

#  the DataFrame with detected events should now contain the following columns:
#  text_id, page_id, name, onset, offset, duration, amplitude, peak_velocity, location
dataset.events[0]

### Plotting the Saccadic Main Sequence

The saccadic main sequence describes the characteristic relationship between a saccade’s amplitude and its peak velocity: larger saccades tend to be faster, following a nonlinear, saturating curve.

It is commonly used to validate saccade detection and assess data quality, since deviations from the expected pattern can indicate recording errors or atypical oculomotor behavior.

In [ ]:
# show the first three event dataframes.
# note that you can adjust the styling of the plot, e.g. setting a low
# alpha value allows you to see overlapping data points
for event_df in dataset.events[:3]:
    pm.plotting.main_sequence_plot(
        event_df,
        title='Main sequence plot for '
        f'text {event_df[0, "text_id"]}, '
        f'page {event_df[0, "page_id"]}',
        alpha=0.5,
        color='green',
        marker='x',
        marker_size=30,
    )